In [3]:
#Created by RK
#Code with smile


#Building deep neural network step by step.
#In this code, I will build important function which will help us in creating deep neural network. 
#So, basically every Deep Neural Network is divided into 5 steps:
# 1. Initialization of Parameters
# 2. Build the forward propogation
# 3. Computer Cost function
# 4. Backward propogation
# 5. Update Parameters
#packages
import numpy as np
import matplotlib.pyplot as plt
from dnn_helper_utils import relu,sigmoid,relu_backward,sig_backward


In [22]:
#Part 1: Initialize Paramters
# layer_dims -- array list containing the dimensions of each layer
def initialize_paramters_deep(layer_dims):
    L = len(layer_dims)
    parameters = {}
    for i in range(1,L):
        parameters["W"+str(i)] = np.random.randn(layer_dims[i],layer_dims[i-1])*0.01
        parameters["b"+str(i)] = np.zeros((layer_dims[i],1))

        assert(parameters["W"+str(i)].shape == (layer_dims[i],layer_dims[i-1]))
        assert(parameters["b"+str(i)].shape == (layer_dims[i],1))
            
    return parameters

In [24]:
def linear_forward(A,W,b):       
    z = np.dot(W,A) + b
    assert(z.shape == (W.shape[0],A.shape[1]))
    cache = (A,W,b)
    return z, cache

In [25]:
def linear_activation_forward(A_prev,W,b,activation):
    z, linear_cache = linear_forward(A_prev,W,b)
    if activation == "sigmoid":
        A,activation_cache = sigmoid(z)
        
    if activation == "relu":
        A,activation_cache = relu(z)
        
    cache = (linear_cache,activation_cache)
    return A,cache

In [28]:
# X -- data (input size, no. of examples)
# parameters -- output of initialize_parameters_deep
#AL last layer output
def l_layer_forward(X,parameters):
    caches = {}
    L = len(parameters)
    A = X
    for i in range(1,L):
        A_prev = A
        A,cache = linear_activation_forward(A_prev,parameters["W"+str(i)],parameters["b"+str(i)],"relu")
        caches.append(cache)
        
    AL,cache = linear_activation_forward(A,parameters["W"+str(L)],parameters["b"+str(L)],"sigmoid")
    caches.append(cache)
    
    assert(AL.shape == (1,X.shape[1]))
    return AL,caches
        

In [31]:
#AL -- probablity vector of label predictions
#Y -- true label
def compute_cost(AL,Y):
    m = Y.shape[1]
    cost = -np.sum(np.dot(Y,np.log(AL.T))+np.dot(1-Y,np.log(1-AL.T)))/m
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    return cost
    

In [32]:
def linear_backward(dZ,cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = (1. / m) * np.dot(dZ, A_prev.T) 
    db = (1. / m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)

    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [33]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":
        dZ = sig_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

In [34]:
def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)

    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    current_cache = caches[-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation="sigmoid")

    
    for l in reversed(range(L-1)):
        current_cache = caches[l] 
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 2)], current_cache, activation="relu")
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

In [35]:
def update_parameters(parameters, grads, learning_rate):
    
    L = len(parameters)

    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
        
    return parameters